# Question Answering System
### TOPIC: Animals

In [1]:
from time import sleep
import pandas as pd

In [2]:
class LLM:
    def __init__(self, api_key):
        import google.generativeai as genai
        genai.configure(api_key=api_key) # API Key


        self.model = genai.GenerativeModel('gemini-pro')
        self.generation_config=genai.types.GenerationConfig(candidate_count=1, max_output_tokens=500, temperature=0.1, top_p=.9)
    
    def generate(self, inp):
        '''Generates output using Google API, given the input.'''
        try:
            response = self.model.generate_content(inp+"Answer in one sentences.", generation_config=self.generation_config)
        except:
            return 'Failed to fetch data from API'
        return response.text


In [3]:
api_key = '' # Add your Google API key here.
data_loc = './movies_metadata.csv'
df = pd.read_csv(data_loc)

/tmp/ipykernel_11608/3235250996.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_loc)


In [4]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
ds = df[['budget','original_language','original_title','overview','popularity','release_date','vote_average', 'revenue', 'runtime', 'genres']]
ds = ds.rename(columns={'vote_average': 'rating'})
ds['original_title'] = ds['original_title'].apply(lambda x: x.lower())

In [6]:
ds

,budget,original_language,original_title,overview,popularity,release_date,rating,revenue,runtime,genres
0,30000000,en,toy story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,7.7,373554033.0,81.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,65000000,en,jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,6.9,262797249.0,104.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,0,en,grumpier old men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,6.5,0.0,101.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,16000000,en,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,6.1,81452156.0,127.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,0,en,father of the bride part ii,Just when George Banks has recovered from his ...,8.387519,1995-02-10,5.7,76578911.0,106.0,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...,...,...,...,...,...,...
45461,0,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,NaN,4.0,0.0,90.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
45462,0,tl,siglo ng pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,9.0,0.0,360.0,"[{'id': 18, 'name': 'Drama'}]"
45463,0,en,betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,3.8,0.0,90.0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,0,en,satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,0.0,87.0,[]


In [8]:
llm = LLM(api_key)

movie = input('Enter movie name: ')
flag = True
if not movie: flag = False

if not ds['original_title'].isin([movie]).any() and flag:
        print('The given movie is not present in the dataset')
        flag = False
print('SELECTED MOVIE:',movie.upper(), end='\n\n')        
while flag:
    query = input()
    if not query: break
    details = ds[ds['original_title'] == movie]
    ans = ''
    for i,j in details.items():
        ans += f'{i}: {j}\n'
    

    prompt_template = f'''
You are a Question Answering system that works on a movie dataset. You are given the user query and the data about the movie. 
Answer the query using the data retrieved. If the answer is not presented in the information, answer that the required answer is not present in the dataset.
If asked to summarise, make a detailed summary containing all the retrieved information. 
If asked questions about the movie, try to find answer of that question from the summary.

Query: {query}

Retrived information: {ans}

Answer: '''
    print('Q:', query)
    print('A:', llm.generate(prompt_template))
    sleep(3)
    print('-'*10)


SELECTED MOVIE: JUMANJI

Q: Tell me about the movie
A: Jumanji is a 1995 American adventure film directed by Joe Johnston, starring Robin Williams, Bonnie Hunt, Kirsten Dunst, Bradley Pierce, and David Alan Grier.
----------
Q: What is its rating
A: The rating of the movie is 6.9.
----------
Q: What is its revenue
A: The revenue of the movie is $262,797,249.
----------
Q: Who is elon musk
A: The provided information does not contain any information about elon musk.
----------
Q: What are the genres of jumanji?
A: The genres of Jumanji are Adventure, Fantasy, and Family.
----------
